# Flotorch SDK - Memory User Guide

This notebook provides a comprehensive guide for using the **FlotorchMemory** and **FlotorchAsyncMemory** classes. These classes allow you to store, retrieve, update, and delete conversational memories.

## Table of Contents
1. [Setup](#setup)
2. [Synchronous Memory Operations](#synchronous-memory-operations)
3. [Asynchronous Memory Operations](#asynchronous-memory-operations)
4. [Summary](#summary)

## Setup

First, we need to initialize both memory clients and set up our test configuration. Each memory operation requires specific parameters like user ID, agent ID, and session ID to organize memories properly.

In [1]:
import os
import asyncio
import uuid
from dotenv import load_dotenv
from flotorch.sdk.memory import FlotorchMemory, FlotorchAsyncMemory

# Load environment variables
load_dotenv()

# Configuration - Replace with your actual values
api_key = os.getenv("FLOTORCH_API_KEY")
base_url = os.getenv("FLOTORCH_BASE_URL")
provider_name = "memo-provider"  # Replace with your provider name

if not api_key or not base_url:
    raise ValueError("Please set FLOTORCH_API_KEY and FLOTORCH_BASE_URL environment variables")

In [2]:
# Initialize memory clients
# FlotorchMemory - for synchronous operations
sync_memory = FlotorchMemory(
    api_key=api_key,
    base_url=base_url,
    provider_name=provider_name
)


# Test parameters for sync operations
sync_session_id = str(uuid.uuid4())
sync_user_id = "sync_user_001"
sync_agent_id = "sync_agent_001"
sync_source = "demo_application_sync"

2025-08-04 14:36:37 - flotorch.sdk.memory - INFO - FlotorchMemory initialized (provider_name=memo-provider, base_url=https://qa-gateway.flotorch.cloud)


In [3]:

# FlotorchAsyncMemory - for asynchronous operations
async_memory = FlotorchAsyncMemory(
    api_key=api_key,
    base_url=base_url,
    provider_name=provider_name
)

# Test parameters for async operations (separate from sync)
async_session_id = str(uuid.uuid4())
async_user_id = "async_user_001"
async_agent_id = "async_agent_001"
async_source = "demo_application_async"


2025-08-04 14:36:39 - flotorch.sdk.memory - INFO - FlotorchAsyncMemory initialized (provider_name=memo-provider, base_url=https://qa-gateway.flotorch.cloud)


## Synchronous Memory Operations

The `FlotorchMemory` class provides synchronous methods for memory operations. Use this when you want simple, blocking operations that complete before moving to the next line of code.

### Add Memory (Sync)

The `add()` method stores a new memory entry. 

**Required Parameters:**
- `messages`: List of message objects (must include role and content)
- `userId`: Unique identifier for the user
- `agentId`: Unique identifier for the agent/assistant
- `sessionId`: Unique identifier for the conversation session
- `metadata`: Dictionary containing `source` (mandatory) and other optional metadata

**Returns:** Response object with memory ID and details

In [3]:
# Add a memory entry using sync method
memory_content = "User mentioned they prefer Python over Java for data science projects"

add_response = sync_memory.add(
    messages=[{"role": "user", "content": memory_content}],
    userId=sync_user_id,
    agentId=sync_agent_id,
    sessionId=sync_session_id,
    metadata={
        "source": sync_source,  # Mandatory field
        "category": "preferences",
        "importance": 0.8
    }
)

# Extract memory ID from response for use in other operations
sync_memory_id = add_response["data"][0]["id"]

print("✓ Sync memory added successfully")
print(f"Memory ID: {sync_memory_id}")
print(f"Stored memory: {memory_content}")
print(f"Response structure: {list(add_response.keys())}")

✓ Sync memory added successfully
Memory ID: 990c8107-d52c-4f94-886b-2e65efc4979f
Stored memory: User mentioned they prefer Python over Java for data science projects
Response structure: ['object', 'data']


### Search Memory (Sync)

The `search()` method retrieves memories based on various criteria.

**Required Parameters:**
- `userId`: User ID to search memories for
- `sessionId`: Session ID to search within

**Optional Parameters:**
- `page`: Page number for pagination (default: 1)
- `limit`: Number of results per page (default: 10)
- `categories`: List of categories to filter by
- `keywords`: List of keywords to search for

**Returns:** Response object with array of matching memories

In [4]:
# Search for memories using sync method
search_response = sync_memory.search(
    userId=sync_user_id,
    sessionId=sync_session_id,
    page=1,
    limit=10
)

print("✓ Sync memory search completed")
print(f"Found {len(search_response['data'])} memories")

# Display found memories
for i, memory in enumerate(search_response['data'], 1):
    print(f"\n{i}. Memory ID: {memory['id']}")
    print(f"   Memory: {memory['memory']}")  # Use 'memory' field instead of 'content'
    print(f"   Metadata: {memory.get('metadata', {})}")
    print(f"   Created: {memory.get('createdAt', 'N/A')}")

✓ Sync memory search completed
Found 1 memories

1. Memory ID: 990c8107-d52c-4f94-886b-2e65efc4979f
   Memory: User prefers Python over Java for data science projects
   Metadata: {'tags': [], 'source': 'demo_application_sync', 'category': 'preferences', 'importance': 0.8}
   Created: 2025-08-04T04:54:31.335Z


### Update Memory (Sync)

The `update()` method modifies an existing memory entry.

**Required Parameters:**
- `memory_id`: ID of the memory to update (first parameter)

**Optional Parameters:**
- `content`: New content for the memory
- `metadata`: Updated metadata (will merge with existing metadata)

**Returns:** Response object with updated memory details

In [5]:
# Update the memory content and metadata using sync method
updated_memory_content = "User strongly prefers Python over Java for data science and machine learning projects"

update_response = sync_memory.update(
    sync_memory_id,  # Memory ID is the first parameter
    content=updated_memory_content,
    metadata={
        "source": sync_source,  # Keep mandatory source field
        "category": "preferences", 
        "importance": 0.9,  # Increased importance
        "updated": True,
        "last_modified": "2024-demo"
    }
)

print("✓ Sync memory updated successfully")
print(f"Updated memory: {updated_memory_content}")
print(f"Response contains: {list(update_response.keys())}")

✓ Sync memory updated successfully
Updated memory: User strongly prefers Python over Java for data science and machine learning projects
Response contains: ['object', 'data']


### Delete Memory (Sync)

The `delete()` method permanently removes a memory entry.

**Required Parameters:**
- `memory_id`: ID of the memory to delete

**Returns:** Response object confirming deletion

**⚠️ Warning:** This operation is permanent and cannot be undone.

In [6]:
# Delete the memory using sync method
delete_response = sync_memory.delete(sync_memory_id)

print("✓ Sync memory deleted successfully")
print(f"Deleted memory ID: {sync_memory_id}")
print(f"Delete response: {delete_response}")

✓ Sync memory deleted successfully
Deleted memory ID: 990c8107-d52c-4f94-886b-2e65efc4979f
Delete response: 


## Asynchronous Memory Operations

The `FlotorchAsyncMemory` class provides asynchronous methods for memory operations. Use this when you want non-blocking operations that can run concurrently or when integrating with async applications.

**Note:** All async methods must be called with `await` and the functions must be defined as `async`.

### Add Memory (Async)

The `add()` method stores a new memory entry asynchronously. The parameters are identical to the sync version, but the method must be awaited.

**Required Parameters:**
- `messages`: List of message objects (must include role and content)
- `userId`: Unique identifier for the user
- `agentId`: Unique identifier for the agent/assistant
- `sessionId`: Unique identifier for the conversation session
- `metadata`: Dictionary containing `source` (mandatory) and other optional metadata

**Returns:** Response object with memory ID and details (awaitable)

In [7]:
async def add_async_memory():
    # Add a memory entry using async method
    memory_content = "User is interested in learning TensorFlow for deep learning applications"
    
    add_response = await async_memory.add(
        messages=[{"role": "user", "content": memory_content}],
        userId=async_user_id,
        agentId=async_agent_id,
        sessionId=async_session_id,
        metadata={
            "source": async_source,  # Mandatory field
            "category": "learning",
            "importance": 0.7,
            "technology": "tensorflow"
        }
    )
    
    # Extract memory ID from response
    async_memory_id = add_response["data"][0]["id"]
    
    print("✓ Async memory added successfully")
    print(f"Memory ID: {async_memory_id}")
    print(f"Stored memory: {memory_content}")
    
    return async_memory_id

# Execute the async add operation
async_memory_id = await add_async_memory()

✓ Async memory added successfully
Memory ID: d8bc82bc-af3b-4a76-aa27-bce5f1c828f9
Stored memory: User is interested in learning TensorFlow for deep learning applications


### Search Memory (Async)

The `search()` method retrieves memories asynchronously. Parameters are identical to the sync version.

**Required Parameters:**
- `userId`: User ID to search memories for
- `sessionId`: Session ID to search within

**Optional Parameters:**
- `page`: Page number for pagination (default: 1)
- `limit`: Number of results per page (default: 10)
- `categories`: List of categories to filter by
- `keywords`: List of keywords to search for

**Returns:** Response object with array of matching memories (awaitable)

In [8]:
async def search_async_memory():
    # Search for memories using async method
    search_response = await async_memory.search(
        userId=async_user_id,
        sessionId=async_session_id,
        page=1,
        limit=10
    )
    
    print("✓ Async memory search completed")
    print(f"Found {len(search_response['data'])} memories")
    
    # Display found memories
    for i, memory in enumerate(search_response['data'], 1):
        print(f"\n{i}. Memory ID: {memory['id']}")
        print(f"   Memory: {memory['memory']}")  # Use 'memory' field instead of 'content'
        print(f"   Metadata: {memory.get('metadata', {})}")
        print(f"   Created: {memory.get('createdAt', 'N/A')}")
    
    return search_response

# Execute the async search operation
search_result = await search_async_memory()

✓ Async memory search completed
Found 1 memories

1. Memory ID: d8bc82bc-af3b-4a76-aa27-bce5f1c828f9
   Memory: User is interested in learning TensorFlow for deep learning applications
   Metadata: {'tags': [], 'source': 'demo_application_async', 'category': 'learning', 'importance': 0.7}
   Created: 2025-08-04T04:54:45.000Z


### Update Memory (Async)

The `update()` method modifies an existing memory entry asynchronously. Parameters are identical to the sync version.

**Required Parameters:**
- `memory_id`: ID of the memory to update (first parameter)

**Optional Parameters:**
- `content`: New content for the memory
- `metadata`: Updated metadata (will merge with existing metadata)

**Returns:** Response object with updated memory details (awaitable)

In [9]:
async def update_async_memory():
    # Update the memory content and metadata using async method
    updated_memory_content = "User is actively learning TensorFlow and PyTorch for deep learning and computer vision applications"
    
    update_response = await async_memory.update(
        async_memory_id,  # Memory ID is the first parameter
        content=updated_memory_content,
        metadata={
            "source": async_source,  # Keep mandatory source field
            "category": "learning",
            "importance": 0.9,  # Increased importance
            "technology": "tensorflow,pytorch",
            "updated": True,
            "applications": "computer_vision"
        }
    )
    
    print("✓ Async memory updated successfully")
    print(f"Updated memory: {updated_memory_content}")
    print(f"Response contains: {list(update_response.keys())}")
    
    return update_response

# Execute the async update operation
update_result = await update_async_memory()

✓ Async memory updated successfully
Updated memory: User is actively learning TensorFlow and PyTorch for deep learning and computer vision applications
Response contains: ['object', 'data']


### Delete Memory (Async)

The `delete()` method permanently removes a memory entry asynchronously. Parameters are identical to the sync version.

**Required Parameters:**
- `memory_id`: ID of the memory to delete

**Returns:** Response object confirming deletion (awaitable)

**⚠️ Warning:** This operation is permanent and cannot be undone.

In [10]:
async def delete_async_memory():
    # Delete the memory using async method
    delete_response = await async_memory.delete(async_memory_id)
    
    print("✓ Async memory deleted successfully")
    print(f"Deleted memory ID: {async_memory_id}")
    print(f"Delete response: {delete_response}")
    
    return delete_response

# Execute the async delete operation
delete_result = await delete_async_memory()

✓ Async memory deleted successfully
Deleted memory ID: d8bc82bc-af3b-4a76-aa27-bce5f1c828f9
Delete response: 


## Summary

This guide covered all core memory operations for both synchronous and asynchronous implementations:

### **FlotorchMemory (Synchronous)**
- **`add(messages, userId, agentId, sessionId, metadata)`** - Store new memories
- **`search(userId, sessionId, page, limit, ...)`** - Find existing memories
- **`update(memory_id, content, metadata)`** - Modify memory content/metadata
- **`delete(memory_id)`** - Remove memories permanently

### **FlotorchAsyncMemory (Asynchronous)**
- **`await add(messages, userId, agentId, sessionId, metadata)`** - Store new memories (async)
- **`await search(userId, sessionId, page, limit, ...)`** - Find existing memories (async)
- **`await update(memory_id, content, metadata)`** - Modify memory content/metadata (async)
- **`await delete(memory_id)`** - Remove memories permanently (async)

### **Key Points to Remember:**

1. **Mandatory Fields:**
   - `source` field is required in metadata for all add/update operations
   - `userId`, `agentId`, and `sessionId` are required for memory organization

2. **Data Access:**
   - Use `memory['memory']` to access memory content, not `memory['content']`
   - Response data is typically in `response['data']` array

3. **When to Use Which:**
   - Use **sync** for simple, sequential operations
   - Use **async** for concurrent operations or async applications

4. **Error Handling:**
   - Always wrap operations in try-except blocks for production code
   - Memory IDs are required for get, update, and delete operations